In [3]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
import pandas as pd
import numpy as np
import re
import string 

In [5]:
imdb_prelim = pd.read_excel("IMDB_dataset.xlsx")
imdb_prelim.head()

,review,sentiment
0,I thought this was a wonderful way to spend ti...,positive
1,"Probably my all-time favorite movie, a story o...",positive
2,I sure would like to see a resurrection of a u...,positive
3,"This show was an amazing, fresh & innovative i...",negative
4,Encouraged by the positive comments about this...,negative
